# Transactions

Desejamos construir uma aplicação de empréstimos peer-to-peer simples. Nesta aplicação os usuários podem:
 
- Depositar uma certa quantia em sua conta;
- Sacar da conta, desde que tenham saldo;
- Criar um "emprestimo" (mais como uma linha de credito, realmente) entre dois usuarios
- Pagar (parcial ou totalmente) um emprestimo
- Tomar mais dinheiro ainda de um emprestimo (ou linha de credito)

Mais ainda, temos o requisito de que todas as operações devem ser armazenadas no banco de dados.

Vamos agrupar esses requerimentos como segue:

- Movimentação de conta: depositar ou sacar da conta;
- Criação de empréstimo;
- Operação em empréstimo: adicionar valor (como quando o devedor saca da linha de credito, ou quando os juros são computados), remover valor (como quando o devedor paga alguma quantia).

A modelagem relacional deste problema resultou no seguinte diagrama:

<img src='diagrama.png' alt='Diagrama do modelo relacional' style='width: 500px;'/>


que traduzimos no script de criação da base de dados como segue:

---
```SQL
DROP DATABASE IF EXISTS emprestimos;
CREATE DATABASE emprestimos;
USE emprestimos;

CREATE TABLE usuario (
    id_usuario INT NOT NULL AUTO_INCREMENT,
    nome VARCHAR(80) NOT NULL,
    sobrenome VARCHAR(80) NOT NULL,
    saldo DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    PRIMARY KEY (id_usuario),
    CONSTRAINT c_saldo CHECK (saldo >= 0.0)
);

CREATE TABLE emprestimo (
    id_emprestimo INT NOT NULL AUTO_INCREMENT,
    id_credor INT NOT NULL,
    id_devedor INT NOT NULL,
    valor_atual DECIMAL(30 , 2 ) NOT NULL DEFAULT 0.0,
    data_inicio DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    data_modificação DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP,
    PRIMARY KEY (id_emprestimo),
    CONSTRAINT fk_credor FOREIGN KEY (id_credor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT fk_devedor FOREIGN KEY (id_devedor)
        REFERENCES usuario (id_usuario),
    CONSTRAINT c_valor CHECK (valor_atual >= 0.0)
);

CREATE TABLE operacao (
    id_operacao INT NOT NULL AUTO_INCREMENT,
    id_emprestimo INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_operacao),
    CONSTRAINT fk_emprestimo FOREIGN KEY (id_emprestimo)
        REFERENCES emprestimo (id_emprestimo)
);

CREATE TABLE movimentacao (
    id_movimentacao INT NOT NULL AUTO_INCREMENT,
    id_usuario INT NOT NULL,
    valor DECIMAL(30 , 2 ),
    data_operacao DATETIME NOT NULL DEFAULT CURRENT_TIMESTAMP,
    PRIMARY KEY (id_movimentacao),
    CONSTRAINT fk_usuario FOREIGN KEY (id_usuario)
        REFERENCES usuario (id_usuario)
);

```
---

Rode esse script ('`emprestimos.sql`') para criar a base de dados no MySQL. Vamos também trazer o código de conexão à base de dados:

In [ ]:
import mysql.connector
import os
from functools import partial
from dotenv import load_dotenv

In [ ]:
load_dotenv(override=True)

def get_connection_helper():

    def run_db_query(connection, query, args=None, verbose=True):
        with connection.cursor() as cursor:
            if verbose:
                print('Executando query:')
            cursor.execute(query, args)
            for result in cursor:
                if verbose:
                    print(result)

    connection = mysql.connector.connect(
        host=os.getenv('MD_DB_SERVER'),
        user=os.getenv('MD_DB_USERNAME'),
        password=os.getenv('MD_DB_PASSWORD'),
        database='emprestimos',
    )
    return connection, partial(run_db_query, connection)


connection, db = get_connection_helper()

## Transactions

Uma *transação* é um grupo de operações na base de dados, em uma dada *sessão*. Vamos conhecer o conceito de transação com as atividades a seguir.

### `COMMIT`

Vamos gerar alguns usuários na nossa base:

In [ ]:
db('''
INSERT INTO usuario (id_usuario, nome, sobrenome) VALUES
    (1, 'Juca', 'Silva'), 
    (2, 'Mario', 'Ferreira'), 
    (3, 'Ana', 'Soares'), 
    (4, 'Antonio', 'Reis'), 
    (5, 'Paulo', 'Oliveira')
''')

Vamos verificar se os usuários foram inseridos:

In [ ]:
db('SELECT * FROM usuario')

Excelente, parece que deu tudo certo! Agora vamos verificar diretamente no MySQL Workbench, só para ter certeza:

```SQL
SELECT * FROM usuario;
```

<img src='usuarios_null.png' alt='Nenhum usuario na tabela!' style='width: 400px;'/>

Epa, cade os usuarios?

Para entender o que aconteceu, precisamos compreender melhor como funciona uma conexão com o banco de dados. Quando criamos um objeto de conexão, estamos iniciando uma *sessão* do banco de dados. Em uma sessão as várias operações sendo realizadas configuram uma *transação*. 

As transações não são efetuadas diretamente no armazenamento permanente, a não ser que o banco tenha a propriedade de *'auto-commit'*. Vamos verificar se esse é o caso na nossa conexão:

In [ ]:
print(connection.autocommit)

Você deve ver o valor 'False' acima. Isso significa que devemos explicitamente indicar quando estamos satisfeitos com uma transação, para que os valores inseridos (ou removidos) sejam efetivamente registrados no banco de dados. Isso é feito através do comando `COMMIT`:

In [ ]:
db('COMMIT')

Agora consulte o banco de dados via MySQL Workbench (ou seja, usando uma sessão diferente da nossa):


```SQL
SELECT * FROM usuario;
```

<img src='usuarios_commit.png' alt='Agora tem usuario na tabela!' style='width: 400px;'/>

Pronto, agora temos usuarios no nosso banco de dados!

Outra forma de indicar o 'commit' da transação é usar o método `commit()` do objeto `connection`:

```Python
connection.commit()
```

### Atividade

Crie os usuários 'Carla Nakamura' e 'Maria Fontana'.

### ROLLBACK

Para efeito de testes, vamos inserir mais um usuario:

In [ ]:
db('INSERT INTO usuario (nome, sobrenome) VALUES ("Fernando", "Maia")')

Verificando o resultado da sessão, temos:

In [ ]:
db('SELECT * FROM usuario')

Agora, se executarmos o `COMMIT` a nossa modificação se tornará permanente. Porém suponha que estamos arrependidos, e não queremos prosseguir com o `COMMIT` da transação, o que fazer?

Para abandonar as mudanças da transação atual, usamos o comando `ROLLBACK`:

In [ ]:
db('ROLLBACK')

Verificando o estado do banco de dados na sessão atual temos:

In [ ]:
db('SELECT * FROM usuario')

Observe que o *rollback* só volta até o último *commit*, e a repetição de *rollbacks* NÃO volta mais ainda para o passado! `COMMIT` e `ROLLBACK` são definitivos!

O *rollback* pode ser obtido também com o método `rollback()` do objeto `connection`:

```Python
connection.rollback()
```

### Atividade

Insira o usuario 'Walter Branco' mas não execute um `COMMIT`, e sim um `ROLLBACK`. Verifique que o `ROLLBACK` funcionou.

### `START TRANSACTION`

Para garantir que o banco de dados MySQL não está operando em modo *auto-commit* use o comando `START TRANSACTION`. Este comando garante que estamos iniciando uma transação sem *auto-commit*. Por exemplo:

In [ ]:
db('START TRANSACTION')
db('INSERT INTO usuario (nome, sobrenome) VALUES ("Barack", "Trump")')
db('COMMIT')
db('SELECT * FROM usuario')

Vamos reverter essa inserção em uma outra transação.

In [ ]:
db('START TRANSACTION')
db('''
SELECT id_usuario INTO @id_usuario 
    FROM usuario 
    WHERE nome = 'Barack' AND sobrenome = 'Trump' 
    ORDER BY id_usuario DESC LIMIT 1;
''')
db('DELETE FROM usuario WHERE id_usuario=@id_usuario')
db('COMMIT')
db('SELECT * FROM usuario')

### Usando `COMMIT` e `ROLLBACK` para garantir atomicidade de transações

Vamos agora juntar os vários conceitos em um código Python para adicionar itens na tabela `emprestimo`. 

Como podemos ver no modelo E-R devemos informar os ids do credor e do devedor ao inserir uma nova linha nesta tabela, e estes ids devem ser ids válidos de clientes do nosso sistema. Logo, 
- se tentarmos inserir um emprestimo entre os usuarios 1 e 2 devemos ter sucesso, 
- mas se tentarmos inserir um emprestimo entre os usuarios 1 e 1000 devemos ter uma falha. 

Se ambos os comandos de inserção devem ser executados de modo 'tudo ou nada' (ou executam ambos, ou nenhum), parece que temos um caso ideal para aplicar transações aqui. A idéia é que se tudo acontecer sem problemas devemos executar um `COMMIT`, mas se algo der errado devemos excutar um `ROLLBACK`. Isso parece uma missão para `try`/`catch`!

In [ ]:
try:
    db('START TRANSACTION')
    db('INSERT INTO emprestimo (id_credor, id_devedor) VALUES (1, 2)')
    db('INSERT INTO emprestimo (id_credor, id_devedor) VALUES (1, 1000)')
    db('COMMIT')
except Exception as e:
    print(e)
    db('ROLLBACK')

In [ ]:
db('SELECT * FROM emprestimo')

### Atividade

- Execute os depósitos abaixo:

| usuario        | valor |
|----------------|-------|
| Ana Soares     | 10000 |
| Antonio Reis   | 5000  |
| Paulo Oliveira | 20000 |

Dica: crie uma função `pega_id_usuario(connection, nome, sobrenome)` para achar o id de usuario, e uma função `deposito(connection, id_usuario, valor)` para facilitar a realização dos depósitos. Um depósito requer uma inserção na tabela de movimentações, e um update no saldo.

- Crie funções para consulta de saldo de conta e de valor atual de emprestimo

- Crie os seguintes contratos de emprestimo:


| Credor | Devedor | valor |
|--|--|--|
| Ana Soares | Juca Silva | 1000 |
| Ana Soares | Antonio Reis | 2000 |
| Paulo Oliveira | Juca Silva | 3000 |

Não se esqueça que um contrato de emprestimo requer:
- criação do emprestimo
- operação de emprestimo inicial
    - operação de saque da conta do credor
    - operacao de deposito na conta do devedor
    
Use o comando `SELECT LAST_INSERT_ID()` para pegar o id do último item inserido.

Dica:
- Crie uma função para criar um empréstimo de valor zero, que retorne o id do emprestimo.
- Crie uma função para realizar um saque. Se não for possível fazer o saque, lance uma exceção.
- Crie uma função para efetuar uma operação em um empréstimo dado - isso inclui fazer o saque e o depósito também.

Por hoje é só!

In [ ]:
connection.close()